In [ ]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gensim.models.keyedvectors as word2vec
from sklearn import preprocessing
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
from gensim.models import KeyedVectors, Word2Vec
from nltk.corpus import stopwords
import nltk
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report





nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
root = "/content/drive/My Drive/Ara Dönem Proje Fake News Detection/Verisetleri"
#dataset name
file_name = "dataset_sum_pre.csv"
df = pd.read_csv(f"{root}/{file_name}")
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
max_review_size = 40

## **Prepare the Dataset**

In [ ]:
df['label'] = df['label'].map({'True': 1, 'FAKE': 0})

In [ ]:
le = preprocessing.LabelEncoder()
df['label'] = le.fit_transform(df['label'])


In [ ]:
df.head()

,text,label
0,kanada 20 kişi yatsı namaz cami yap sıra düzen...,1
1,istanbul terör örgüt pkk kck yönelik düzenle o...,1
2,kocaeli kağıt topla bir şahıs can beden bulun ...,1
3,ses türkiye yılbaş 2018 yılbaşı 2018 program h...,1
4,istanbul cumhuriyet başsavcı yap açıkla fetö ü...,1


In [ ]:
# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

In [ ]:
y_train

20469    1
13571    1
9758     1
17329    1
12490    1
        ..
21575    1
5390     1
860      1
15795    0
23654    1
Name: label, Length: 20125, dtype: int64

## Train

In [ ]:
def vectorize_sentence(sentence:str, word_vectors, vector_size=400, stpwrds= stopwords.words('turkish')):
    sentence = re.sub(r"[^\w\s]", " ", sentence.lower()).split()
    words = np.array([word_vectors[word] for word in sentence if word in word_vectors and word not in stpwrds])
    if words.size:
        return np.mean(words, 0)
    else:
        return np.zeros(vector_size,)

def calc_w2v_representations(texts, keyed_vectors_path: str = None, w2v_size: int = 400, w2v_window: int = 2,
                             w2v_min_count: int = 4, w2v_epoch: int = 16):
    
    
    if keyed_vectors_path is not None:
        word_vectors = KeyedVectors.load_word2vec_format(keyed_vectors_path, binary=True)
    else:
        documents = [text.split() for text in texts]
        w2v_model = Word2Vec(vector_size=w2v_size,
                             window=w2v_window,
                             min_count=w2v_min_count,
                             workers=8,
                             sg=1)
        w2v_model.build_vocab(documents)
        w2v_model.train(documents, total_examples=len(
            documents), epochs=w2v_epoch)

        word_vectors = w2v_model.wv

    vectorize = lambda t: vectorize_sentence(t, word_vectors, vector_size=w2v_size)

    return np.array(list(map(vectorize, texts)))

In [ ]:
X_train = calc_w2v_representations(texts = X_train, keyed_vectors_path='/content/drive/My Drive/Ara Dönem Proje Fake News Detection/Verisetleri/Word2Vec/trmodel')
X_test = calc_w2v_representations(texts = X_test, keyed_vectors_path='/content/drive/My Drive/Ara Dönem Proje Fake News Detection/Verisetleri/Word2Vec/trmodel')

In [ ]:
X_train.shape

(20125, 400)

In [ ]:
from imblearn.over_sampling import SMOTE

def smote(x, y):
    smote = SMOTE(sampling_strategy= 1, random_state=42)
    x, y = smote.fit_resample(x, y)
    return x, y

In [ ]:
print(X_train.shape)
print(y_train.shape)

(20125, 400)
(20125,)


In [ ]:
X_train, y_train = smote(X_train, y_train)

In [ ]:
print(X_train.shape)
print(y_train.shape)

(32040, 400)
(32040,)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.53      0.80      0.64      1018
           1       0.94      0.82      0.88      4014

    accuracy                           0.82      5032
   macro avg       0.74      0.81      0.76      5032
weighted avg       0.86      0.82      0.83      5032



/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.75      0.70      1018
           1       0.93      0.90      0.92      4014

    accuracy                           0.87      5032
   macro avg       0.80      0.83      0.81      5032
weighted avg       0.88      0.87      0.87      5032



In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.32      0.94      0.48      1018
           1       0.97      0.49      0.65      4014

    accuracy                           0.58      5032
   macro avg       0.64      0.71      0.56      5032
weighted avg       0.84      0.58      0.62      5032



In [ ]:
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.33      0.58      0.42      1018
           1       0.87      0.70      0.78      4014

    accuracy                           0.68      5032
   macro avg       0.60      0.64      0.60      5032
weighted avg       0.76      0.68      0.71      5032

